In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

/home/ubuntu/Spell_Correction/Spell_Corrections/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
housing=pd.read_csv('/home/ubuntu/DEEP_LEARNING/Tensorflow_Deeplearning_02042018/02-TensorFlow-Basics/cal_housing_clean.csv')
print(housing.shape)
print(housing.columns)
housing.head()

(20640, 7)
Index([u'housingMedianAge', u'totalRooms', u'totalBedrooms', u'population',
       u'households', u'medianIncome', u'medianHouseValue'],
      dtype='object')


,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [3]:
y=housing['medianHouseValue']
X=housing.drop('medianHouseValue',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [4]:
scalar=MinMaxScaler()
scalar.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [5]:
X_train=pd.DataFrame(data=scalar.transform(X_train),columns=X_train.columns,index=X_train.index)
X_test=pd.DataFrame(data=scalar.transform(X_test),columns=X_test.columns,index=X_test.index)

In [6]:
X_train.columns

Index([u'housingMedianAge', u'totalRooms', u'totalBedrooms', u'population',
       u'households', u'medianIncome'],
      dtype='object')

In [7]:
hma=tf.feature_column.numeric_column('housingMedianAge')
tr=tf.feature_column.numeric_column('totalRooms')
tb=tf.feature_column.numeric_column('totalBedrooms')
population=tf.feature_column.numeric_column('population')
hh=tf.feature_column.numeric_column('households')

In [8]:
feat_cols=[hma,tr,tb,population,hh]

In [9]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

In [10]:
model=tf.estimator.DNNRegressor(hidden_units=[6,8,8,8,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbb9159ddd0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/tmp95ytC3', '_save_summary_steps': 100}


In [11]:
model.train(input_fn=input_func,steps=2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp95ytC3/model.ckpt.
INFO:tensorflow:loss = 443284950000.0, step = 1
INFO:tensorflow:global_step/sec: 253.939
INFO:tensorflow:loss = 75507450000.0, step = 101 (0.396 sec)
INFO:tensorflow:global_step/sec: 312.929
INFO:tensorflow:loss = 208125310000.0, step = 201 (0.319 sec)
INFO:tensorflow:global_step/sec: 305.954
INFO:tensorflow:loss = 123832844000.0, step = 301 (0.327 sec)
INFO:tensorflow:global_step/sec: 281.938
INFO:tensorflow:loss = 174908960000.0, step = 401 (0.358 sec)
INFO:tensorflow:global_step/sec: 299.834
INFO:tensorflow:loss = 63476003000.0, step = 501 (0.330 sec)
INFO:tensorflow:global_step/sec: 296.67
INFO:tensorflow:loss = 79590200000.0, step = 601 (0.339 sec)
INFO:tensorflow:global_step/sec: 298.012
INFO:tensorflow:loss = 101779920000.0, step = 701 (0.334 sec)
INFO:tensorflow:global_step/sec: 307.249
INFO:tensorflow:loss = 49745050000.0, step = 801 (0.327 sec)
INFO:tensorflow:

In [12]:
pred_inpu_eval=tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=10,num_epochs=1,shuffle=False)

In [13]:
prediction=model.predict(pred_inpu_eval)
prediction=list(prediction)
prediction=[pred['predictions'] for pred in prediction]
prediction[0:5]

INFO:tensorflow:Restoring parameters from /tmp/tmp95ytC3/model.ckpt-2000


[array([188320.14], dtype=float32),
 array([211792.08], dtype=float32),
 array([271335.1], dtype=float32),
 array([181705.61], dtype=float32),
 array([211224.89], dtype=float32)]

In [14]:
mean_squared_error(y_test,prediction)**0.5

114277.91413553532

In [15]:
r2_score(y_test,prediction)

0.019996858003300977